In [16]:
import numpy as np
import pandas as pd

import pickle

import scipy
import scipy.sparse as sp

import scripts.training.Dataset_split as spl

In [17]:
PROJECT_FOLDER = "../.."
INPUT_DATA_FOLDER = "data/final_data"
MODEL_FOLDER = "model"
FINAL_TRAIN_MATRIX = "final_train_matrix_v1.pkl"
FINAL_DATETIMES = "final_datetimes_v1"
FINAL_Y = "final_y_v1"
TUNING_FOLDER = f"{MODEL_FOLDER}/tuning"

In [18]:
X = pickle.load(open(f"{PROJECT_FOLDER}/{INPUT_DATA_FOLDER}/{FINAL_TRAIN_MATRIX}", "rb"))
X

<191715x9026 sparse matrix of type '<class 'numpy.float64'>'
	with 129877897 stored elements in Compressed Sparse Row format>

In [19]:
y = pickle.load(open(f"{PROJECT_FOLDER}/{INPUT_DATA_FOLDER}/{FINAL_Y}.pkl", "rb"))
y

0         0
167500    0
159608    0
151693    0
143371    0
         ..
135512    0
119083    0
51952     0
16353     0
191714    0
Name: is_alarm, Length: 191715, dtype: int64

In [ ]:
df_dt = pickle.load(open(f"{PROJECT_FOLDER}/{INPUT_DATA_FOLDER}/{FINAL_DATETIMES}.pkl", "rb"))

In [25]:
print(df_dt.head(30))

                  datetime
0      2022-02-26 00:00:00
167500 2022-02-26 00:00:00
159608 2022-02-26 00:00:00
151693 2022-02-26 00:00:00
143371 2022-02-26 00:00:00
135513 2022-02-26 00:00:00
7906   2022-02-26 00:00:00
126982 2022-02-26 00:00:00
119084 2022-02-26 00:00:00
110697 2022-02-26 00:00:00
102871 2022-02-26 00:00:00
95071  2022-02-26 00:00:00
86863  2022-02-26 00:00:00
78090  2022-02-26 00:00:00
68415  2022-02-26 00:00:00
16354  2022-02-26 00:00:00
60189  2022-02-26 00:00:00
51953  2022-02-26 00:00:00
44112  2022-02-26 00:00:00
36193  2022-02-26 00:00:00
175685 2022-02-26 00:00:00
183770 2022-02-26 00:00:00
25430  2022-02-26 00:00:00
25431  2022-02-26 01:00:00
167501 2022-02-26 01:00:00
95072  2022-02-26 01:00:00
143372 2022-02-26 01:00:00
51954  2022-02-26 01:00:00
86864  2022-02-26 01:00:00
119085 2022-02-26 01:00:00


In [21]:
df_dt.loc[df_dt['datetime']==pd.to_datetime("2022-02-26 00:00:00")].index


Int64Index([     0, 167500, 159608, 151693, 143371, 135513,   7906, 126982,
            119084, 110697, 102871,  95071,  86863,  78090,  68415,  16354,
             60189,  51953,  44112,  36193, 175685, 183770,  25430],
           dtype='int64')

In [40]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit
import scripts.training.Dataset_split as spl
import os

In [48]:
TUNING_PATH = f"{PROJECT_FOLDER}/{TUNING_FOLDER}"
MODEL_NAME = "logistic_regression_v7"
print(MODEL_NAME)

logistic_regression_v7


In [49]:
def train_model (n_splits, max_iter, solver, min_split, max_split):
    split = 1
    tss = TimeSeriesSplit(n_splits=n_splits)
    MIN_ITERATION = min_split
    MAX_ITERATION = max_split
    RANDOM_STATE = 1
    #getting indexes for df_all_features from splitting df_datetime

    for train_index, test_index in zip(*spl.split_dataset(df_dt, tss)):
        if(split<MIN_ITERATION):
            split+=1
            continue
        if(split>MAX_ITERATION):
            break

        model=LogisticRegression(max_iter=max_iter, random_state=RANDOM_STATE, solver=solver, verbose=1, multi_class='ovr', n_jobs=-1)

        X_train=X[train_index]
        X_test=X[test_index]
        y_train=y[train_index]
        y_test=y[test_index]

        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)

        print(f"{MODEL_NAME}_split{split}: {score}")
        FOLDER=f"{TUNING_PATH}/{MODEL_NAME}"
        if not os.path.exists(FOLDER):
            os.makedirs(FOLDER)
        with open(f"{TUNING_PATH}/{MODEL_NAME}/{MODEL_NAME}_{solver}_split{split}.pkl", 'wb') as handle: pickle.dump(model, handle)
        split+=1

In [50]:
train_model(n_splits=4, max_iter=500, min_split=4, max_split=4, solver='saga')

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


max_iter reached after 623 seconds
logistic_regression_v7_split4: 0.8593155893536122


C:\Users\IvarY\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 10.4min finished


NameError: name 'MODEL_NAME' is not defined